In [30]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [31]:
import numpy as np
import pandas as pd
import optuna as op
from sklearn.model_selection import ParameterSampler
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Utils.createURM import tryURM
from sklearn.model_selection import train_test_split
from Utils.createICM import createICM
from Utils.combineURMICM import combine

### URM

In [32]:
dataset = pd.read_csv('../Input/interactions_and_impressions.csv')
URM = tryURM(dataset)
ICM = createICM()

C:\Users\Mike\AppData\Local\Temp\ipykernel_1488\3000734721.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Input/interactions_and_impressions.csv')
C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Utils\createICM.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  URM = pd.read_csv('../Input/interactions_and_impressions.csv')


In [25]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

In [26]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 762 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 331 ( 0.8%) Users that have less than 1 test interactions


In [27]:
URMICMCombined = combine(ICM, URM_train)

### SLIM

In [28]:
recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URMICMCombined.T, verbose=False)

def objective(trial):
    alpha = trial.suggest_float("alpha", 0.00001, 0.1)
    l1_ratio = trial.suggest_float("l1_ratio", 0.000001, 0.1)
    topK = trial.suggest_float("topK", 300, 400)
    recommender.fit(alpha=alpha, l1_ratio=l1_ratio, topK=int(topK))

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    return result_dict.loc[10]["MAP"]

In [29]:
study = op.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

[I 2022-11-08 12:34:05,180] A new study created in memory with name: no-name-b53662ea-0e1b-40ac-972f-5cb1f478a1fa

  1%|          | 168/24507 [00:21<41:43,  9.72it/s]

KeyboardInterrupt: 

In [ ]:
alpha = study.best_params['alpha']
l1_ratio = study.best_params['l1_ratio']
topK = study.best_params['topK']

In [33]:
recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URMICMCombined.T, verbose=False)
recommender.fit(alpha=0.06747829810332745, l1_ratio=0.0005493724398243842, topK=362)
result_dict, _ = evaluator_test.evaluateRecommender(recommender)



  0%|          | 0/24507 [00:00<?, ?it/s]

  0%|          | 8/24507 [00:08<7:12:52,  1.06s/it]

  0%|          | 40/24507 [00:09<1:17:17,  5.28it/s]

  0%|          | 72/24507 [00:09<35:57, 11.33it/s]  

  0%|          | 104/24507 [00:15<49:32,  8.21it/s]

  1%|          | 136/24507 [00:16<34:53, 11.64it/s]

  1%|          | 168/24507 [00:18<33:04, 12.26it/s]

  1%|          | 200/24507 [00:20<30:09, 13.44it/s]

  1%|          | 232/24507 [00:24<37:12, 10.88it/s]

  1%|          | 264/24507 [00:24<26:30, 15.25it/s]

  1%|          | 296/24507 [00:28<31:55, 12.64it/s]

  1%|▏         | 328/24507 [00:31<33:38, 11.98it/s]

  1%|▏         | 360/24507 [00:31<25:06, 16.03it/s]

  2%|▏         | 392/24507 [00:33<26:05, 15.40it/s]

  2%|▏         | 424/24507 [00:38<37:12, 10.79it/s]

  2%|▏         | 456/24507 [00:39<28:15, 14.18it/s]

  2%|▏         | 488/24507 [00:41<27:16, 14.67it/s]

  2%|▏         | 520/24507 [00:45<35:21, 11.31it/s]

  2%|▏         | 552/24507 [00:46<27:19, 14.61it/s]


EvaluatorHoldout: Processed 41298 (100.0%) in 33.60 sec. Users per second: 1229


In [34]:
result_dict

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.057003,0.077355,0.067285,0.027259,0.036556,0.177157,0.083572,0.061718,0.36503,0.218124,...,0.992049,0.362127,0.992049,0.025624,9.118886,0.995109,0.049084,0.67286,3.703427,0.318744


In [21]:
import json

resultParameters = result_dict.to_json(orient="records")
parsed = json.loads(resultParameters)

with open("../Hyperparameter Tuning/SLIM_ElasticNet/logs/SLIMElasticNetRecommender_logs.json" + recommender.RECOMMENDER_NAME + "_logs.json", 'w') as json_file:
    #json.dump(study.best_params, json_file, indent=4)
    json.dump(parsed, json_file, indent=4)

In [24]:
result_dict.iloc[0]['MAP']

0.02116256670654663

## Course To Fine

In [ ]:
grid_size = 100
TUNE_ITER = 10
num_epochs = 2
worse_score = 0

init_param_grid = {'l1_ratio': [i for i in range(0, 1)],
                   'topK': [i for i in range(10, 500)]
                  }

new_param_grid = init_param_grid.copy()
best_params_dict = {'score':worse_score,'params':[]}
tried_params_list = []

for epoch in range(num_epochs):

    # List of sampled hyperparameter combinations will be used for random search
    param_list = list(ParameterSampler(new_param_grid, n_iter=TUNE_ITER,random_state=0))

    # Searching the Best Parameters with Random Search
    rs_results_dict = {}
    for tune_iter in range(TUNE_ITER):
        # Get the set of parameter for this iteration
        strategy_params = param_list[tune_iter]

        recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
        recommender.fit(alpha=0.0001, l1_ratio=strategy_params['l1_ratio'], topK=strategy_params['topK'])
        results, _ = evaluator_validation.evaluateRecommender(recommender)
        results = results.loc[10]['MAP']

        rs_results_dict[tuple(strategy_params.values())] = {'score':results}

        if results > best_params_dict['score']:
            best_params_dict['score'] = results
            best_params_dict['params'] = list(strategy_params.values())

    # Save the results in dataframe and sort it based on score
    df_rs_results = pd.DataFrame(rs_results_dict).T.reset_index()
    df_rs_results.columns = list(strategy_params.keys()) + ['score']
    df_rs_results = df_rs_results.sort_values(['score'],ascending=False).head(num_epochs-epoch)

    # If the best score from this epoch is worse than the best score,
    # then append the best hyperaparameters combination to this epoch dataframe
    if df_rs_results['score'].iloc[0] < best_params_dict['score']:
        new_row_dict = {}
        new_row_dict['score'] = best_params_dict['score']
        for idx, key in enumerate(init_param_grid):
            new_row_dict[key] = best_params_dict['params'][idx]

        df_rs_results = df_rs_results.append(new_row_dict,ignore_index=True)
        df_rs_results = df_rs_results.sort_values(['score'],ascending=False).head(num_epochs-epoch)

    display(df_rs_results)
    print(df_rs_results.head(1).T.to_dict())

    # Get the worse and best hyperparameter combinations
    df_rs_results_min = df_rs_results[df_rs_results['score']>worse_score].min(axis=0)
    df_rs_results_max = df_rs_results[df_rs_results['score']>worse_score].max(axis=0)

    # Generate new hyperparameter space based on current worse and best hyperparameter combinations
    for key in init_param_grid:
        if isinstance(init_param_grid[key][0],int):
            new_param_grid[key] = np.unique([i for i in range(int(df_rs_results_min[key]), int(df_rs_results_max[key])+1)])
        elif isinstance(init_param_grid[key][0],float):
            new_param_grid[key] = np.unique(np.linspace(df_rs_results_min[key], df_rs_results_max[key], grid_size))
        else:
            new_param_grid[key] = init_param_grid[key]

    # Decrease the tuning iteration for random search
    TUNE_ITER = int(TUNE_ITER - epoch * TUNE_ITER/num_epochs)

In [ ]:
best_params_dict